# 写在前面

这个笔记介绍NLFFF数据集和pynlfff工具包基本使用方法。


## 说明

工具包分为数据集建造和产品使用两部分。


### 数据集建造

+ 数据集建造目前有cpu版本，可以在 [pynlfff github](https://github.com/deepsolar/pynlfff/tree/master/pynlfff) 找到数据集建造相关程序



### 三维磁场数据产品使用

+ 产品使用分为基本使用和基于工具包的使用两部分。

    + 我们将工具包中基本使用方法提炼出来了，以帮助不方便下载完整工具包和希望借鉴思路通过其他语言实现的研究者。 

    + 基于工具包的使用我们提供了一些稍微高阶功能，如可视化等，后续将追加更多功能。


> notebook可以[点击这里下载](https://github.com/deepsolar/pynlfff/blob/main/test/guide/Dataset_nlfff_and_pynlfff_base_use_zh.ipynb)，也可以[点击这里在colab中打开](https://drive.google.com/file/d/1Nx3xW4N8B2-W5FybV_fjP_ngoxJDp0HI/view?usp=sharing)， html版本可以在[这里查看](https://github.com/deepsolar/pynlfff/blob/main/test/guide/Dataset_nlfff_and_pynlfff_base_use_zh.html)





# 数据集和标签




## 下载数据集摘要信息

In [ ]:
# 下载数据集摘要信息
!wget https://figshare.com/ndownloader/files/39406012 -O dataset_info.zip

In [ ]:
# 解压数据集摘要信息
!unzip dataset_info.zip

## 下载标签信息

In [ ]:
!wget https://figshare.com/ndownloader/files/39406015 -O dataset_label.zip

In [ ]:
!unzip dataset_label.zip

## 下载数据库归档信息

In [ ]:
!wget https://figshare.com/ndownloader/files/39406063 -O dataset_archive.zip

In [ ]:
!unzip dataset_archive.zip

In [ ]:
nlfff_path_dataset_summary="nlfff-dataset-info"
nlfff_path_dataset_label="nlfff-dataset-flare-label"
nlfff_path_dataset_archive="nlfff-dataset-and-flare-label-db-archive"

# 使用数据集摘要信息

## 读取配置

In [ ]:
import os
import json

# 读取配置文件
nlfff_path_dataset_summary="nlfff-dataset-info"
config_path=os.path.join(nlfff_path_dataset_summary,"config.json")
with open(config_path,"r") as f:
    config_ds=json.loads(f.read())

In [ ]:
config_ds

## 读取原始头文件信息

In [ ]:
import pandas
import os

raw_info_path=os.path.join(
  nlfff_path_dataset_summary,
  config_ds["raw_summary_info_path"]
)
raw_info=pandas.read_csv(raw_info_path)

In [ ]:
raw_info

## 读取产品文件信息

记录三维磁场数据库产品的一些基本信息

In [ ]:
product_info_path=os.path.join(
  nlfff_path_dataset_summary,
  config_ds["product_summary_info_path"]
)
product_info=pandas.read_csv(product_info_path)

In [ ]:
product_info

# 使用标签信息

## 读取配置

In [ ]:
import os
import json

# 读取配置文件
nlfff_path_dataset_label="nlfff-dataset-flare-label"
config_path=os.path.join(nlfff_path_dataset_label,"config.json")
with open(config_path,"r") as f:
    config_label=json.loads(f.read())

In [ ]:
config_label

## 标签信息

In [ ]:
label_info_path=os.path.join(
    nlfff_path_dataset_label,
    config_label["label_info_path"]
)
label_info=pandas.read_csv(label_info_path)

In [ ]:
label_info

## 耀斑信息

记录耀斑详细信息，如果需要耀斑详细信息可以执行此步。

In [ ]:
flare_info_path=os.path.join(
    nlfff_path_dataset_label,
    config_label["flare_info_path"]
)

flare_info=pandas.read_csv(flare_info_path)

In [ ]:
flare_info

# 使用数据库归档信息

In [ ]:
import json
import sqlite3
import os

## 读取配置

In [ ]:
import os
import json

# 读取配置文件
nlfff_path_dataset_archive="nlfff-dataset-and-flare-label-db-archive"
config_path=os.path.join(nlfff_path_dataset_archive,"config.json")
with open(config_path,"r") as f:
    config_db=json.loads(f.read())

In [ ]:
config_db

In [ ]:
archive_db_path=os.path.join(
    nlfff_path_dataset_archive,
    config_db["archive_db_path"]
)
archive_db_path

## 归档文件的基本介绍

归档文件存储格式为sqlite数据库文件，包括一下信息：

+ 样本原始fits文件头文件信息(time,noaa number)
+ 样本的产品信息（nx,ny,nz）
+ 样本标签（CMX flare label）
+ 耀斑信息（start time）

### 基本使用

查询表及表信息

In [ ]:
# 查询存在哪些表
query_sql="select name from sqlite_master where type='table' order by name; "
table_list=[]
with sqlite3.connect(archive_db_path) as conn:
   c = conn.cursor()
   cursor = c.execute(query_sql)
   for table in cursor:
      print(table)
      table_list.append(table[0])
print(table_list)

In [ ]:
# 查询某个表有哪些列
table_name="nlfff_archive"
query_sql="PRAGMA table_info({});".format(table_name)
column_list=[]
with sqlite3.connect(archive_db_path) as conn:
   c = conn.cursor()
   cursor = c.execute(query_sql)
   for column in cursor:
      print(column)
      column_list.append(column[1])
print(column_list)

In [ ]:
# 创建表名和列名的映射字典

ktablename_vcolumnlist=dict()

for table_name in table_list:
   print(table_name)
   # table_name="nlfff_archive"
   query_sql="PRAGMA table_info({});".format(table_name)
   column_list=[]
   with sqlite3.connect(archive_db_path) as conn:
      c = conn.cursor()
      cursor = c.execute(query_sql)
      for column in cursor:
         # print(column)
         column_list.append(column[1])
   # print(column_list)
   ktablename_vcolumnlist[table_name]=column_list

In [ ]:
ktablename_vcolumnlist

### 查询表数据

In [ ]:
# 样本原始fits文件头文件信息(time,noaa number)

table_name="nlfff_raw"
with sqlite3.connect(archive_db_path) as conn:
   c = conn.cursor()

   query_sql="SELECT *  from {}".format(table_name)
   cursor = c.execute(query_sql)
   for row in cursor:
      row_data_list=list(row)
      row_column_name_list=ktablename_vcolumnlist[table_name]
      row_dict=dict(zip(row_column_name_list,row_data_list))
      print(row_dict)
      break


In [ ]:
# 样本的产品信息（nx,ny,nz）

table_name="nlfff_archive"
with sqlite3.connect(archive_db_path) as conn:
   c = conn.cursor()

   query_sql="SELECT *  from {}".format(table_name)
   cursor = c.execute(query_sql)
   for row in cursor:
      row_data_list=list(row)
      row_column_name_list=ktablename_vcolumnlist[table_name]
      row_dict=dict(zip(row_column_name_list,row_data_list))
      print(row_dict)
      break


In [ ]:
# 样本标签（CMX flare label）

table_name="nlfff_flare_label"
with sqlite3.connect(archive_db_path) as conn:
   c = conn.cursor()

   query_sql="SELECT *  from {}".format(table_name)
   cursor = c.execute(query_sql)
   for row in cursor:
      row_data_list=list(row)
      row_column_name_list=ktablename_vcolumnlist[table_name]
      row_dict=dict(zip(row_column_name_list,row_data_list))
      print(row_dict)
      break


In [ ]:
# 耀斑信息（start time）

table_name="flare_info"
with sqlite3.connect(archive_db_path) as conn:
   c = conn.cursor()

   query_sql="SELECT *  from {}".format(table_name)
   cursor = c.execute(query_sql)
   for row in cursor:
      row_data_list=list(row)
      row_column_name_list=ktablename_vcolumnlist[table_name]
      row_dict=dict(zip(row_column_name_list,row_data_list))
      print(row_dict)
      break


### 高级查询

+ python 脚本可以使用 pandas + 某个引擎，如 pandas+[sqlalchemy](https://www.sqlalchemy.org)
+ 可视化程序，如[dbeaver](https://dbeaver.io)
+ 其他语言工具包链接sqlite


# 使用单个样本（基本使用）

## 基本读取

### 获取存储路径和时间

使用一个样本最少需要下面信息：

+ 非线性无力场数据Bout.bin

+ Bout.bin对应的nx，ny，nz

+ 样本的harp number 和时间

In [ ]:
# 在数据集摘要信息里面有大于或等于一个样本，
# 我们以这个样本为例进行说明

# 以0号样本为例，读取已经下载样本的`harpnum_trec`和路径
nlfff_path_dataset_summary="nlfff-dataset-info"
sample_num=0
sample_harpnum_trec=config_ds["demo_sample"][sample_num]["harpnum_trec"]
sample_path=os.path.join(
    nlfff_path_dataset_summary,
    config_ds["demo_sample"][sample_num]["path"])

In [ ]:
# 样本的harp number 和时间
sample_harpnum_trec

In [ ]:
# Bout path
sample_bout_path=os.path.join(sample_path,"Bout.bin")
sample_bout_path

### 获取样本产品文件信息（nx ny nz）

In [ ]:
# 获取样本产品文件信息

sample_product_info=product_info[ product_info["harpnum_trec"]==sample_harpnum_trec ]

In [ ]:
sample_product_info

In [ ]:

# 样本的最大计算等级 
sample_bout_maxlevel=int(sample_product_info["bout_maxlevel"])
# 样本的nx，ny，nz分别为
sample_nx=int(sample_product_info["grid_x"])
sample_ny=int(sample_product_info["grid_y"])
sample_nz=int(sample_product_info["grid_z"])
# 样本的identifiers为
sample_identifiers=int(sample_product_info["identifiers"])

### 读取数据

In [ ]:
# 根据 nx ny nz 读取数据
import numpy

nx=sample_nx
ny=sample_ny
nz=sample_nz

np_dtype_str=r"<d"
bin_path=os.path.join(sample_path,"Bout.bin")

# https://numpy.org/doc/stable/reference/generated/numpy.memmap.html
nlfff_data = numpy.memmap(sample_bout_path,
                dtype=numpy.dtype(np_dtype_str),
                offset=0,
                shape=(3, nx, ny, nz),
                order='C')  

In [ ]:
nlfff_data.shape

## 获取关联信息

### 获取样本原始头文件信息

In [ ]:
# 获取样本原始头文件信息

sample_raw_info=raw_info[ raw_info["harpnum_trec"]==sample_harpnum_trec ]

In [ ]:
sample_raw_info

### 获取标签信息

In [ ]:
# 获取某个样本的标签

# sample_harpnum_trec="997.20111030_142400_TAI"
sample_label=label_info[label_info["harpnum_trec"]==sample_harpnum_trec]

In [ ]:
# 这个样本的所有标签信息
sample_label

## 获取二级关联信息

### 标签对应耀斑信息

In [ ]:
# 获取某个有耀斑的正样本样本

sample_harpnum_trec_pos="997.20111030_142400_TAI"
sample_label_pos=label_info[label_info["harpnum_trec"]==sample_harpnum_trec_pos]

In [ ]:
# 这个样本的所有标签信息
sample_label_pos

In [ ]:
# 进一步也可以获取信息的耀斑信息

# 以上面样本12小时内发生耀斑信息为例
# 获取上面样本12小时内发生耀斑的id
sample_h12_flare_id=int(sample_label_pos["h12_flare_id"])

In [ ]:
sample_h12_flare_id

In [ ]:
# 对应耀斑详细信息
sample_h12_flare_detail_info=flare_info[flare_info["deeps_flare_id"]==sample_h12_flare_id]

In [ ]:
sample_h12_flare_detail_info

# 使用单个样本（pynlfff工具包）

## 工具包初始化

### 安装pynlfff工具包




In [ ]:
# 安装最新工具包
!pip uninstall pynlfff -y
!pip install pynlfff

# 导入工具包
import pynlfff


### 安装工具包依赖

In [ ]:
# 因为整个工具包依赖比较多，所以没有设置在下载工具包时全部安装

# 可以使用下面命令按照需要按照工具包
# 创建对象
checker=pynlfff.RCheck()
# 查看有哪些模块需要安装依赖
checker.h()

In [ ]:
# 使用4安装和Product有关的依赖
checker.check(4)

# 使用0即按照全部依赖
checker.check(0)

## 数据读取



### 获取存储路径和时间

使用一个样本最少需要下面信息：

+ 非线性无力场数据Bout.bin

+ Bout.bin对应的nx，ny，nz

+ 样本的harp number 和时间

In [ ]:
# 在数据集摘要信息里面有大于或等于一个样本，
# 我们以这个样本为例进行说明

# 以0号样本为例，读取已经下载样本的`harpnum_trec`和路径
nlfff_path_dataset_summary="nlfff-dataset-info"
sample_num=0
sample_harpnum_trec=config_ds["demo_sample"][sample_num]["harpnum_trec"]
sample_path=os.path.join(
    nlfff_path_dataset_summary,
    config_ds["demo_sample"][sample_num]["path"])

In [ ]:
# 样本的harp number 和时间
sample_harpnum_trec

In [ ]:
# Bout path
sample_bout_path=os.path.join(sample_path,"Bout.bin")
sample_bout_path

### 获取样本产品文件信息（nx ny nz）

In [ ]:
# 获取样本产品文件信息

sample_product_info=product_info[ product_info["harpnum_trec"]==sample_harpnum_trec ]
sample_product_info

In [ ]:

# 样本的最大计算等级 
sample_bout_maxlevel=int(sample_product_info["bout_maxlevel"])
# 样本的nx，ny，nz分别为
sample_nx=int(sample_product_info["grid_x"])
sample_ny=int(sample_product_info["grid_y"])
sample_nz=int(sample_product_info["grid_z"])
# 样本的identifiers为
sample_identifiers=int(sample_product_info["identifiers"])

### 读取数据

In [ ]:
from pynlfff.pyproduct import file

# 创建读取对象
r=file.NlfffFile()



In [ ]:
r = file.NlfffFile()


bout_data = r.read_bin(sample_bout_path, nx=sample_nx,ny=sample_ny,nz=sample_nz)
print(bout_data.shape)
# print(s)

In [ ]:
bout_data

## 格式转换

### 转HDF

In [ ]:
r = file.NlfffFile()

h5_path = sample_bout_path.replace(".bin",".h5")

r.tran_bin2hdf5(sample_bout_path, h5_path, nx=sample_nx,ny=sample_ny,nz=sample_nz, overwrite=True)

## 可视化(dev 注意：可能与colab不兼容)

### 三维切割绘制

#### 创建绘图对象



In [ ]:
%matplotlib inline
from pynlfff.pyplot import plot3d_cut


In [ ]:
# 方法1

d3_drawer = plot3d_cut.NlfffPlotD3CutCake()

In [ ]:
# 方法2
# 其中可以创建时传入`matplotlib.pyplot.figure`的设置参数，如

d3_drawer = plot3d_cut.NlfffPlotD3CutCake(figsize=(6, 6))


In [ ]:
# 方法3
# 或者字典形式传入参数

di_config = {
	"figsize": (6, 6)
}
d3_drawer = plot3d_cut.NlfffPlotD3CutCake(**di_config)

#### 设置绘图对象属性（可选）

In [ ]:
# 手动设置色调范围


d3_drawer.colormap_set_max=40
# 设置最大显示值

d3_drawer.colormap_set_max=-30
# 设置最小显示值

In [ ]:
# 自动设置色调范围


d3_drawer.colormap_auto_value=True
# 自动求的绘图切面中的最大和最小值

d3_drawer.colormap_auto_zip=0.8
# 只有colormap_auto_value为True才生效，表示在自动取最值上再进行压缩，压缩比例，取值范围(0,1]，1表示不压缩

d3_drawer.colormap_auto_mirror=True
# 表示取最值绝对值最大的值为最大值，其相反数为最小值，时得到的调色板关于0对称，注意如果最大值和最小值同为正或同为负，则不生效

In [ ]:
# 越界值显示

d3_drawer.colormap_out_range_display=True
# 默认设为True，越界值按照离得最近的那个最值取；为False，越界值不显示

In [ ]:
# 透明度

d3_drawer.colormap_alpha=None
# 设置None则不透明

d3_drawer.colormap_alpha=0.8
# 设置整体透明度，取值范围(0,1)

d3_drawer.colormap_alpha="auto"
# 默认，auto自动透明，即绝对值越小透明程度越高


#### 设置切面交线（可选）

In [ ]:
# 设置切面边界

d3_drawer.cut_line_edges = None
#不绘制边界线

d3_drawer.cut_line_edges = dict(color='0.6', linewidth=0.6, zorder=1e3) 
# 默认，设置边界线样式

In [ ]:
# 设置切面交线

d3_drawer.cut_line_cross = None
# 不绘制切面交叉线

d3_drawer.cut_line_cross = dict(color='0.4', linewidth=1, zorder=1e3)
# 默认，设置切面交叉线样式

#### 加载数据

In [ ]:
array_data=bout_data.copy()
d3_drawer.load_data_array(array_data)

In [ ]:
import numpy as np
array_data = np.random.uniform(-5, 5, size=(3, 100,50,40))

In [ ]:
d3_drawer.load_data_array(array_data)

#### 添加绘图切面

In [ ]:
d3_drawer.add_cut(B="Bx", N="Nx", cut_num=None, cut_percent=0.5)

#### 执行绘图操作

In [ ]:
# 使用下面命令开始执行绘图操作，即处理数据，生成图片元素对象，
# 但是如果不展示或者保存下来，仍然无法可视化。

d3_drawer.run_cut()

#### 保存或展示图片

In [ ]:
# savefig其中传入参数需要满足`matplotlib.pyplot.savefig()`方法的参数要求。
picture_path="NlfffPlotD3CutCake.png"
d3_drawer.savefig(picture_path)

In [ ]:
# 展示图片
d3_drawer.show()

### 三维体绘制

In [ ]:
import os


In [ ]:
%matplotlib inline

is_with_gui=False

if is_with_gui:
  from pynlfff.pyplot import plot3d_body
  import numpy as np
  out_dir="./"
  try:
    array_data = np.random.uniform(-5, 5, size=(3, 100,50,40))
    dataBout=array_data#.copy()
    result=plot3d_body.main_draw_3d_body(
        dataBout,out_dir,axis_name=['bx','by','bz']
    )
  except BaseException as e:
    print(e)
print("VTK 绘制依赖gui")

# 更多样本获取方法

## 方法1 通过官方网站查询下载

访问<https://nlfff.dataset.deepsolar.space/>的下载页面，找到**在线查询下载**相关网站

## 方法2 通过identifiers跳转在线查看样本

In [ ]:
# sample identifiers可以从产品信息表里面获得

sample_identifiers=7327201812200936003
# 其中 7327(harp number)20181220(date)093600(time)3(Max grid level).
sample_url="https://identifiers.org/nlfff:{}".format(sample_identifiers)
print("访问 {} 查看样本详细信息及下载".format(sample_url))

## 方法3 在线申请邮寄硬盘拷贝

对于需要使用大量样本的研究者，我们推荐使用硬盘邮寄的方式。

可以访问访问<https://nlfff.dataset.deepsolar.space/>的**联系**页面，通过邮件沟通邮寄地址收件人等具体细节。